# Metode lokalne optimizacije bez ograničenja

Matematička optimizacija bez ograničenja se vezuje za nalaženje minimuma ili maksimuma neke funkcije. Formalno govoreći, za neku funkciju $f: D \rightarrow \mathbb{R}$, $D \subseteq \mathbb{R}^n$, problem minimizacije predstavlja nalaženje vrednosti $\min_{x \in D} f(x)$, a problem maksimizacije nalaženje vrednosti $\max_{x \in D} f(x)$. Problemi minimizacije i maksimizacije su ekvivalentni u smislu da nalaženje minimuma funkcije $f$ predstavlja ujedno nalaženje maksimuma funkcije $-f$, zbog čega će u nastvaku biti reči samo o minimizaciji. Sva rešenja $x \in D$ su dopustiva, a optimalno se smatra onim za koje je vrednost $f(x)$ najmanja kod problema minimizacije, odnosno najveća kod problema maksimizacije.

U nastavku će biti pomenute neke metode matematičke optimizacije prvog i drugog reda bez ograničenja. Metode prvog reda vrše optimizaciju na osnovu vrednosti funkcije i njenog prvog izvoda, a kod metoda drugog reda, u obzir se uzimaju i vrednosti drugog izvoda. Metoda prvog reda koju ćemo pomenuti će biti gradijentni spust, a od metoda drugog reda biće pomenut Njutnov algoritam za minimizaciju. Biće prikazana i biblitečka podrška za minimizaciju funkcije za razne slučajeve.

In [1]:
import numpy as np
from scipy import optimize as opt

## Gradijentni spust

Gradijentni spust (eng. *gradient descent*) započinje pretragu od proizvoljno izabrane tačke $x_0$, a zatim se, kroz određen broj iteracija, naredna tačka bira krećući se u smeru suprotnom od smera gradijenta funkcije $f$.  Ako je u $k$-tom koraku izabrana tačka $x_k$, naredna tačka je određena sa $x_{k+1} = x_k - \alpha \nabla f(x_k)$. Kod funkcije jedne promenljive, umesto gradijenta $\nabla f(x_k)$, u obrascu figuriše prvi izvod $f'(x_k)$. Parametar algoritma $\alpha$ je takozvani korak učenja (engl. learning rate) i može biti fiksiran ili se menjati kroz iteracije. U praksi, ako je njegova vrednost previše mala, sporija je konvergencija ka lokalnom minimumu, a ako je previše velika, algoritam često obilazi minimum. Algoritam se zaustavlja nakon dostignutog broja iteracija ili kada vrednost $|f(x_{k+1})-f(x_k)|$ postane manja od unapred zadate vrednosti $\epsilon$.

In [2]:
def gradient_descent(f, gradient, x0, alpha, eps, max_iterations):
    x = x0
    
    # u svakoj iteraciji
    for iteration in range(max_iterations):
        # azuriramo vrednost tacke optimuma
        x_new = x - alpha * gradient(x)
        
        # proveravamo da li je ispunjen zaustavni kriterijum
        if np.abs(f(x_new) - f(x)) < eps:
            break
           
        # vrsimo pripremu za narednu iteraciju
        x = x_new
        
    # algoritam je iskonvergirao ako je zaustavljen uslovom abs(f(x_new) - f(x)) < eps
    converged = iteration != max_iterations
    
    # pripremamo i vracamo rezultat
    result = {}
    result['number_of_iterations'] = iteration
    result['x'] = x_new
    result['converged'] = converged
    
    return result

Kao primer, izvršimo metodom gradijentnog spusta nalaženje lokalnog optimuma funkcije $$f(x, y) = \frac{1}{2}(x^2 + 10y^2)$$ za početnu vrednost $(3, 5)$. Gradijent funkcije je $\nabla f(x, y) = (f'_x(x, y), f'_y(x, y)) = (x, 10y)$. Neka je $\alpha = 0.1$, $\epsilon = 0.01$ i maksimalan broj iteracija 1000.

In [3]:
def f(x):
    return 0.5 * (x[0]**2 + 10*x[1]**2)

In [4]:
def gradient(x):
    return np.array([x[0], 10*x[1]])

In [5]:
x0 = np.array([3, 5])
alpha = 0.1
eps = 0.01
max_iterations = 1000

In [6]:
gradient_descent(f, gradient, x0, alpha, eps, max_iterations)

{'number_of_iterations': 22,
 'x': array([0.26588814, 0.        ]),
 'converged': True}

Zaključujemo da je dostignuta konvergencija u lokalnom optimumu u tački $(0.26588814, 0)$. 

Napomenimo da se pomoću gradijentnog spusta dostiže lokalni optimum. U nekim slučajevima, lokalni optimum može biti i globalni optimum, ali to algoritmom ne može da se garantuje. Slično važi za ostale metode koje će biti pomenute.

Na sledećem <a href='http://www.benfrederickson.com/numerical-optimization/'> linku </a> možemo ispratiti simulaciju gradijentnog spusta za nekoliko različitih funkcija i načine na koje veličina koraka učenja utiče na brzinu konvergencije. <img src='assets/gradient_descent.png' width='600px'>

### Linijska pretraga

Linijska pretraga (eng. *line search*) predstavlja nalaženje maksimalne vrednosti parametra $\alpha$ za koju će biti izvršena minimizacija funkcije duž nekog pravca $p$. Preciznije, za neki početni vektor $x$ i funkciju $f: D \rightarrow \mathbb{R}$, traži se vrednost parametra $\alpha$ tako da $f(x+\alpha p)$ bude minimalno. Pritom se ne zahteva da se na ovaj način nađe lokalni minimum, već da se za odabranu vrednost parametra $\alpha$ vrednost funkcije $f$ dovoljno smanji. Zbog toga se nakon linijske pretrage često može primeniti neki drugi algoritam minimizacije. 

Sam algoritam za pronalaženje ove vrednosti je iterativnog karaktera. Počevši od neke velike vrednosti parametra $\alpha$ vrednost se postepeno smanjuje. Kod `backtracking` linijske pretrage koja predstavlja varijantu osnovnog algoritma tokom iteracija vrednost parametra $\alpha$ se smanjuje tako što se množi parametrom $\tau \in (0, 1)$.

In [7]:
def backtracking_line_search(f, x, p, tau):
    # pocetna vrednost parametra
    alpha = 1
    
    # pronalazi se vrednost parametra alfa takva da smanjuje vrednost funkcije f duz odabranog pravca p
    while f(x + alpha * p) >= f(x):
        alpha *= tau
    
    
    return alpha

Ako posmatramo funkciju iz prethodnog primera, istu početnu tačku, pravac suprotan pravcu gradijenta u početnoj tački i vrednost parametra $\tau=0.6$ dobijamo vrednost koraka. 

In [8]:
backtracking_line_search(f, (3,5), -gradient((3,5)), 0.6)

0.1296

Često se za kriterijum zaustavljanja prethodnog algoritma razmatra zadovoljivost Armijo pravila (engl. Armijo-Goldstein condition) koje je za pogodno izabran parametar $\beta \in (0, 1)$ oblika 

$$f(x + \alpha p) \le f(x) + \beta \alpha p^T \nabla f(x).$$ 



Ovim pravilom se zapravo zahteva da se odabere takva vrednost koraka kojom se vrednost funkcije smanjuje u zadovoljavajućoj meri određenoj parametrom $\beta$. 

In [9]:
def armijo_rule_line_search(f, x, p, gradval, tau, beta):
    alpha = 1
    while f(x + alpha * p) >= f(x) + alpha * beta * np.dot(p.T, gradval):
        alpha *= tau
    return alpha

In [10]:
armijo_rule_line_search(f, (3,5), -gradient((3,5)), gradient((3,5)), 0.6, 0.8)

0.027993599999999997

Sada ćemo modifikovati implementaciju gradijentnog spusta tako da se koristi linijska pretraga sa Armijo pravilom za određivanje parametra $\alpha$.

In [11]:
def gradient_descent_armijo(f, gradient, x0, eps, max_iterations):
    x = x0
    
    # u svakoj iteraciji 
    for iteration in range(max_iterations):
        
        # odredjujemo vrednost koraka
        alpha = armijo_rule_line_search(f, x, -gradient(x), gradient(x), 0.6, 0.8)
        
        # azuriramo vrednost tacke optimuma
        x_new = x - alpha * gradient(x)
        
        # proveravamo da li je ispunjen zaustavni kriterijum
        if np.abs(f(x_new) - f(x)) < eps:
            break
            
        # vrsimo pripremu za narednu iteraciju
        x = x_new

    # algoritam je iskonvergirao ako je zaustavljen uslovom abs(f(x_new) - f(x)) < eps
    converged = iteration != max_iterations
    
    # pripremamo i vracamo rezultat
    result = {}
    result['number_of_iterations'] = iteration
    result['x'] = x_new
    result['converged'] = converged
    
    return result

In [12]:
gradient_descent_armijo(f, gradient, (3,5), 0.01, 1000)

{'number_of_iterations': 20,
 'x': array([ 0.19401419, -0.0027003 ]),
 'converged': True}

U paketu `scipy.optimize` postoji ugrađena funkcija `line_search` koja implementira linijsku pretragu ali tako da zadovoljava takozvane Volfove uslove (engl. <a href='https://en.wikipedia.org/wiki/Wolfe_conditions'> Wolfe condition </a>). Argumenti ove funkcije su: funkcija koja se posmatra, gradijent, početna tačka i pravac duž koga se ide. U nizu povratnih vrednosti značajan je prvi element, koji predstavlja traženu vrednost parametra $\alpha$. [Ovde](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.line_search.html) možete pogledati detaljniji opis pomenute bibliotečke funkcije.

In [13]:
alpha = opt.line_search(f, gradient, (3,5), -gradient((3,5)))[0]
alpha

0.10032388340197529

## Njutnova metoda

Njutnova metoda pripada metodama lokalne optimizacije drugog reda bez ograničenja, budući da osim informacije o vrednostima funkcije i njenom prvom izvodu, koristi i vrednosti drugog izvoda funkcije. Kod funkcija više promenljivih, koristi se hesijan tj. matrica drugih izvoda funkcije. Pomoću Njutnove metode se, takođe, nalazi lokalni optimum funkcije. 

U slučaju funkcije jedne promenljive, krećući se od početne tačke $x_0$, u svakoj iteraciji se ažurira vrednost tačke. Ako je nakon $k$ iteracija razmatrana tačka bila $x_k$, nova tačka $x_{k+1}$ se dobija pomoću obrasca

$$x_{k+1} = x_k - \frac{f'(x_k)}{f''(x_k)}.$$

U slučaju funkcija više promenljivih, novi vektor $x_{k+1}$ se dobija od vektora $x_k$ pomoću

$$x_{k+1} = x_k - [\nabla^2 f(x_k)]^{-1} \nabla f(x_k),$$

gde je $\nabla f(x_k)$ gradijent, a $\nabla^2 f(x_k)$ hesijan funkcije $f$ u tački $x_k$.

In [14]:
from numpy import linalg as LA

Sledećim kodom možemo implementirati Njutnovu metodu.

In [15]:
def newton_method(f, gradient, hessian, x0, eps, max_iterations):
    x = x0
    for iteration in range(max_iterations):
        x_new = x - np.dot(LA.inv(hessian(x)), gradient(x))
        if np.abs(f(x_new) - f(x)) < eps:
            break
        x = x_new
    
    converged = iteration != max_iterations
    
    # pripremamo i vracamo rezultat
    result = {}
    result['number_of_iterations'] = iteration
    result['x'] = x_new
    result['converged'] = converged
    
    return result

Neka je Njutnovom metodom potrebno minimizovati funkciju 

$$f(x, y) = \frac{1}{2}(x-1)^2 + (x^2 - y)^2.$$

u tački $(2, 2)$. Njen gradijent je

$$\nabla f(x, y) = \begin{bmatrix}f'_x(x, y) & f'_y(x, y)\end{bmatrix} = \begin{bmatrix} x-1+4x(x^2-y) & 2(y-x^2)\end{bmatrix},$$

a hesijan

$$\nabla^2 f(x, y) = \begin{bmatrix}f''_{xx}(x, y) & f''_{xy}(x, y) \\ f''_{yx}(x, y) & f''_{yy}(x, y) \end{bmatrix} = \begin{bmatrix} 12x^2-4y+1 & -4x \\ -4x & 2 \end{bmatrix}.$$

Uzmimo da je $\epsilon = 10^{-4}$ i broj iteracija 1000.

In [16]:
def f(x):
    return 0.5*(x[0]-1)**2 + (x[0]**2-x[1])**2 

def gradient(x):
    return np.array([x[0]-1+4*x[0]*(x[0]**2-x[1]), 2*(x[1]-x[0]**2)])

def hessian(x):
    return np.array([
        [12*x[0]**2-4*x[1]+1, -4*x[0]],
        [-4*x[0], 2]
    ])

In [17]:
newton_method(f, gradient, hessian, (2, 2), 0.0001, 1000)

{'number_of_iterations': 4,
 'x': array([1.00000006, 1.00000011]),
 'converged': True}

Funkcija dostiže lokalni minimum u tački (1, 1).

U okviru paketa `scipy.optimize` funkcija `fmin_ncg` vrši optimizaciju Njutnovom metodom. Njena prva tri argumenta su, redom, funkcija čiji minimum se traži, početna tačka i gradijent, a parametrom `fhess` se zadaje vrednost hesijana. Po završetku funkcija ispisuje status optimizacije i vraća pronađeni minimum.

In [18]:
opt.fmin_ncg(f, (2,2), gradient, fhess=hessian)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 11
         Hessian evaluations: 9


array([1., 1.])

Biblioteka stavlja na raspolaganje i neke kvazi-Njutnove metode optimizacije koje ne zahtevaju izračunavanje hesijana, već koriste njegove numerički jeftinije aproksimacije. Među ovim metodama su `fmin_bfgs` i `fmin_l_bfgs_b` i obe se naslanjaju na `Broyden, Fletcher, Goldfarb & Shanno` predlog aproksimacije hesijana.

In [19]:
opt.fmin_bfgs(f, (2,2), gradient)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 8
         Function evaluations: 9
         Gradient evaluations: 9


array([1.00000582, 1.00001285])

In [20]:
opt.fmin_l_bfgs_b(f, (2,2), gradient)

(array([1.00000005, 1.00000009]),
 1.441767747301186e-15,
 {'grad': array([ 1.02331202e-07, -2.59299369e-08]),
  'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 17,
  'nit': 16,
  'warnflag': 0})

Druga u nizu funkcija je pogodna za velike dimenzije jer koristi aproksimaciju hesijana niskog ranga i memorijski je efikasna.

### Zadaci za vežbu:

1. Koristeći navedene metode, odrediti minimum funkcije $f(x_0, x_1)=2\cdot x_0^2+3\cdot x_1^2$. Za početnu tačku uzeti $(1, 1)$.

2. U oblasti mašinskog učenja gradijentni spust se koristi za minimizaciju funkcije gubitka. Možete istražiti sledeći <a href='https://ruder.io/optimizing-gradient-descent/'>post</a> i pročitati više o varijacijama i popravkama ovog algoritma.